In [ ]:
# 이 코드를 사용해 오프라인 대회 장려상 수상 -> 9.385초
import math

def reward_function(params):
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    heading = params['heading']
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    all_wheels_on_track = params['all_wheels_on_track']
    speed = params['speed']

    SPEED_THRESHOLD = 1

    marker_1 = 0.1 * track_width
    marker_2 = 0.20 * track_width
    marker_3 = 0.30 * track_width
    marker_4 = 0.40 * track_width
    marker_5 = 0.5 * track_width

    if distance_from_center <= marker_1 and all_wheels_on_track:
        reward = 3.0
    elif distance_from_center <= marker_2 and all_wheels_on_track:
        reward = 2.5
    elif distance_from_center <= marker_3 and all_wheels_on_track:
        reward = 1.5
    elif distance_from_center <= marker_4 and all_wheels_on_track:
        reward = 1
    elif distance_from_center <= marker_5 and all_wheels_on_track:
        reward = 0.5
    else:
        reward = 1e-3

    if speed < SPEED_THRESHOLD:
        reward = reward + 0.5
    else:
        reward = reward + 1.0

    if params["all_wheels_on_track"] and params["steps"] > 0:
        reward += ((params["progress"] / params["steps"]) * 100) + (params["speed"]**2)
    else:
        reward = 0.01

    center_variance = params["distance_from_center"] / params["track_width"]

    left_lane = []  # Fill in the waypoints
    center_lane = [0, 1, 2, 3, 7,8,9,10,11,12,17,18,19,20,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,44,45,46,47,48,49,50,51,52,53,54,55]  # Fill in the waypoints
    right_lane = [4,5,6,16,13,14,15,16,21,22,42,43]  # Fill in the waypoints

    fast = [0, 1, 2, 3, 7,8,9,10,11,12,17,18,19,20,23,24,25,26,27,28,29,30,31,32,33,34,44,45,46,47,48,49,50,51,52,53,54,55]  # 3
    moderate = [4,5,6,16,13,14,15,16,21,22,35,36,37,38,39]  # 2
    slow = [40,41,42,43]  # 1

    if params["all_wheels_on_track"]:
        reward += 10
    else:
        reward -= 10

    if params["closest_waypoints"][1] in left_lane and params["is_left_of_center"]:
        reward += 10
    elif params["closest_waypoints"][1] in right_lane and not params["is_left_of_center"]:
        reward += 10
    elif params["closest_waypoints"][1] in center_lane and center_variance < 0.4:
        reward += 10
    else:
        reward -= 10

    if params["closest_waypoints"][1] in fast:
        if params["speed"] > 1.5:
            reward += 10
        else:
            reward -= 10
    elif params["closest_waypoints"][1] in moderate:
        if 1 < params["speed"] <= 1.5:
            reward += 10
        else:
            reward -= 10
    elif params["closest_waypoints"][1] in slow:
        if params["speed"] <= 1:
            reward += 10
        else:
            reward -= 10

    next_point = waypoints[closest_waypoints[1]]
    prev_point = waypoints[closest_waypoints[0]]
    track_direction = math.atan2(next_point[1] - prev_point[1], next_point[0] - prev_point[0])
    track_direction = math.degrees(track_direction)
    direction_diff = abs(track_direction - heading)

    DIRECTION_THRESHOLD = 10.0

    if direction_diff > 180:
        direction_diff = 360 - direction_diff

    if direction_diff > DIRECTION_THRESHOLD:
        reward *= 0.5

    return float(reward)